## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)
working_op_df = application_df

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 500
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts[classification_counts > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value = 1000
classifications_to_replace = application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() < cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
dummy_application_df = pd.get_dummies(application_df)
dummy_application_df.head(5)

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [10]:
# Split our preprocessed data into our features and target arrays
X = dummy_application_df.drop('IS_SUCCESSFUL', axis=1)
y = dummy_application_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [11]:
print(y.value_counts())

1    18261
0    16038
Name: IS_SUCCESSFUL, dtype: int64


In [12]:
X.shape

(34299, 43)

In [13]:
X_train.shape

(25724, 43)

In [14]:
X_test.shape

(8575, 43)

In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [16]:
X_train_scaled.shape

(25724, 43)

## Compile, Train and Evaluate the Model

In [17]:
# Define the model
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation='relu', input_dim=X_train_scaled.shape[1]))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
# Train the model
history = nn.fit(X_train_scaled, y_train, epochs=5, verbose=2, batch_size=X_train_scaled.shape[1])

Epoch 1/5
599/599 - 3s - loss: 0.5725 - accuracy: 0.7205 - 3s/epoch - 5ms/step
Epoch 2/5
599/599 - 2s - loss: 0.5541 - accuracy: 0.7285 - 2s/epoch - 3ms/step
Epoch 3/5
599/599 - 2s - loss: 0.5519 - accuracy: 0.7308 - 2s/epoch - 3ms/step
Epoch 4/5
599/599 - 2s - loss: 0.5501 - accuracy: 0.7308 - 2s/epoch - 3ms/step
Epoch 5/5
599/599 - 2s - loss: 0.5483 - accuracy: 0.7318 - 2s/epoch - 4ms/step


In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=1, batch_size=1)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 [==============================] - 30s 3ms/step - loss: 0.5537 - accuracy: 0.7319
Loss: 0.5537430047988892, Accuracy: 0.7318950295448303


In [21]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity.h5')

## Optimization Method 1 - Add/Drop Columns

In [22]:
from sklearn.linear_model import Lasso, Ridge
from sklearn.metrics import mean_squared_error

In [23]:
# Create a Lasso model
lasso_model = Lasso(alpha=0.1)  # Adjust the alpha parameter for regularization strength
lasso_model.fit(X_train, y_train)

lasso_predictions = lasso_model.predict(X_test)
lasso_rmse = mean_squared_error(y_test, lasso_predictions, squared=False)
print("Lasso RMSE:", lasso_rmse)

# Create a Ridge model
ridge_model = Ridge(alpha=0.1)  # Adjust the alpha parameter for regularization strength
ridge_model.fit(X_train, y_train)

ridge_predictions = ridge_model.predict(X_test)
ridge_rmse = mean_squared_error(y_test, ridge_predictions, squared=False)
print("Ridge RMSE:", ridge_rmse)

Lasso RMSE: 0.49911599627875314
Ridge RMSE: 0.44266081278506914


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=4.66431e-22): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


In [24]:
lasso_coeffs = lasso_model.coef_
ridge_coeffs = ridge_model.coef_
print(lasso_coeffs)
print(ridge_coeffs)

[-0.00000000e+00 -5.13954297e-11  0.00000000e+00  0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00]
[-1.25353504e-01 -6.01701915e-12  2.72383785e-02  3.34714270e-01
 -3.24613353e-01 -1.97367280e-01 -1.92793099e-01  3.71190125e-01
  1.25851844e-01 -4.57555942e-02 -9.84652905e-02 -2.72769593e-01
  1.20924148e-03  8.00968535e-02 -3.46404295e-02  2.13625927e-01
  1.24779998e-02  2.11766076e-02 -1.2293

In [34]:
coefficients_df = pd.DataFrame(columns=["Column", "Lasso Coefficient", "Ridge Coefficient", "Drop_Potential"])

for i in range(len(lasso_coeffs)):
    column_name = X.columns[i]
    lasso_coefficient = lasso_coeffs[i]
    ridge_coefficient = ridge_coeffs[i]
    if lasso_coeffs[i] == 0 and (ridge_coeffs[i] < -0.1 or ridge_coeffs[i] > 0.1):
        drop = 1
    else:
        drop = 0
    row_data = {"Column": column_name, "Lasso Coefficient": lasso_coefficient, "Ridge Coefficient": ridge_coefficient, "Drop_Potential": drop}
    coefficients_df = pd.concat([coefficients_df, pd.DataFrame(row_data, index=[0])])

coefficients_df

,Column,Lasso Coefficient,Ridge Coefficient,Drop_Potential
0,STATUS,-0.000000e+00,-1.253535e-01,1
0,ASK_AMT,-5.139543e-11,-6.017019e-12,0
0,APPLICATION_TYPE_Other,0.000000e+00,2.723838e-02,0
0,APPLICATION_TYPE_T10,0.000000e+00,3.347143e-01,1
0,APPLICATION_TYPE_T19,-0.000000e+00,-3.246134e-01,1
0,APPLICATION_TYPE_T3,-0.000000e+00,-1.973673e-01,1
0,APPLICATION_TYPE_T4,-0.000000e+00,-1.927931e-01,1
0,APPLICATION_TYPE_T5,0.000000e+00,3.711901e-01,1
0,APPLICATION_TYPE_T6,0.000000e+00,1.258518e-01,1
0,APPLICATION_TYPE_T7,0.000000e+00,-4.575559e-02,0


In [66]:
working_op_df['SPECIAL_CONSIDERATIONS'].value_counts()

N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64

In [67]:
Test_Index_DF_0 = working_op_df[working_op_df['SPECIAL_CONSIDERATIONS'] == "Y"]
print(len(Test_Index_DF_0))
print(len(Test_Index_DF_0[Test_Index_DF_0['IS_SUCCESSFUL'] == 1]))
print(len(Test_Index_DF_0[Test_Index_DF_0['IS_SUCCESSFUL'] == 0]))
Test_Index_DF_0.head(5)

27
17
10


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
1374,T3,Independent,C2000,Preservation,Association,1,0,Y,5000,0
2928,T3,Independent,C1000,Preservation,Trust,0,25000-99999,Y,7287,1
6056,T3,Independent,Other,Preservation,Trust,1,0,Y,5000,1
6805,T3,Independent,C1000,Preservation,Trust,1,25000-99999,Y,5000,1
7747,T3,Independent,C1000,Preservation,Trust,1,0,Y,5000,0


In [68]:
working_op_df['INCOME_AMT'].value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [69]:
working_op_df['STATUS'].value_counts()

1    34294
0        5
Name: STATUS, dtype: int64

In [70]:
# Expriment and drop columns, 'EIN' and 'NAME'.
working_op_df = working_op_df.drop(['SPECIAL_CONSIDERATIONS', 'STATUS'], axis=1)

In [71]:
cutoff_value = 50
application_types_to_replace = working_op_df['APPLICATION_TYPE'].value_counts()[working_op_df['APPLICATION_TYPE'].value_counts() < cutoff_value].index.tolist()

for app in application_types_to_replace:
    working_op_df['APPLICATION_TYPE'] = working_op_df['APPLICATION_TYPE'].replace(app,"Other")

In [72]:
cutoff_value = 100
classifications_to_replace = working_op_df['CLASSIFICATION'].value_counts()[working_op_df['CLASSIFICATION'].value_counts() < cutoff_value].index.tolist()

for cls in classifications_to_replace:
    working_op_df['CLASSIFICATION'] = working_op_df['CLASSIFICATION'].replace(cls,"Other")

In [73]:
dummy_working_op_df = pd.get_dummies(working_op_df)
dummy_working_op_df.head(5)

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,108590,1,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,5000,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,6692,1,0,0,0,1,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,142590,1,0,0,0,1,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [74]:
X1 = dummy_working_op_df.drop('IS_SUCCESSFUL', axis=1)
y1 = dummy_working_op_df['IS_SUCCESSFUL']

X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size=0.25, random_state=1)

scaler = StandardScaler()

X_scaler1 = scaler.fit(X_train1)

X_train_scaled1 = X_scaler1.transform(X_train1)
X_test_scaled1 = X_scaler1.transform(X_test1)

In [75]:
# Define the model
nn_op1 = tf.keras.models.Sequential()

# First hidden layer
nn_op1.add(tf.keras.layers.Dense(units=80, activation='relu', input_dim=X_train_scaled1.shape[1]))

# Second hidden layer
nn_op1.add(tf.keras.layers.Dense(units=30, activation='relu'))

# Output layer
nn_op1.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn_op1.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 80)                3520      
                                                                 
 dense_13 (Dense)            (None, 30)                2430      
                                                                 
 dense_14 (Dense)            (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [76]:
nn_op1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history1 = nn_op1.fit(X_train_scaled1, y_train1, epochs=100, verbose=2, batch_size=X_train_scaled1.shape[1])

Epoch 1/100
599/599 - 2s - loss: 0.5716 - accuracy: 0.7192 - 2s/epoch - 3ms/step
Epoch 2/100
599/599 - 1s - loss: 0.5550 - accuracy: 0.7294 - 1s/epoch - 2ms/step
Epoch 3/100
599/599 - 2s - loss: 0.5526 - accuracy: 0.7308 - 2s/epoch - 3ms/step
Epoch 4/100
599/599 - 2s - loss: 0.5511 - accuracy: 0.7305 - 2s/epoch - 4ms/step
Epoch 5/100
599/599 - 3s - loss: 0.5490 - accuracy: 0.7331 - 3s/epoch - 6ms/step
Epoch 6/100
599/599 - 2s - loss: 0.5480 - accuracy: 0.7313 - 2s/epoch - 3ms/step
Epoch 7/100
599/599 - 1s - loss: 0.5475 - accuracy: 0.7319 - 964ms/epoch - 2ms/step
Epoch 8/100
599/599 - 1s - loss: 0.5469 - accuracy: 0.7345 - 972ms/epoch - 2ms/step
Epoch 9/100
599/599 - 2s - loss: 0.5461 - accuracy: 0.7330 - 2s/epoch - 3ms/step
Epoch 10/100
599/599 - 1s - loss: 0.5466 - accuracy: 0.7349 - 1000ms/epoch - 2ms/step
Epoch 11/100
599/599 - 1s - loss: 0.5458 - accuracy: 0.7340 - 957ms/epoch - 2ms/step
Epoch 12/100
599/599 - 1s - loss: 0.5445 - accuracy: 0.7344 - 976ms/epoch - 2ms/step
Epoch 13/

## Optimization Method 2 - Altering Model

In [ ]:
# Model creation function for ease of coding later
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    activation = hp.Choice('activation', ['relu', 'tanh', 'sigmoid', 'leaky_relu', 'elu', 'selu'])

    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=120,
        step=5), activation=activation, input_dim=X_train.shape[1]))
    
    for i in range(hp.Int('num_layers', 2, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=120,
            step=5),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [ ]:
# Install & import the kerastuner library
!pip install keras_tuner
import keras_tuner as kt

In [ ]:
tuner = kt.Hyperband(
    create_model,
    objective="accuracy",
    max_epochs=5,
    hyperband_iterations=2)

In [ ]:
tuner.search(X_train_scaled,y_train,epochs=5,validation_data=(X_test_scaled,y_test), batch_size=X_train_scaled.shape[1])

In [ ]:
top_hyper = tuner.get_best_hyperparameters(5)
for param in top_hyper:
    print(param.values)

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_op = tf.keras.models.Sequential()

# Take optimized hyperparams and make a model
nn_op.add(tf.keras.layers.Dense(units=param.values['first_units'], activation=param.values['activation'], input_dim=X_train_scaled.shape[1]))
for i in range(param.values['num_layers']):
    nn_op.add(tf.keras.layers.Dense(units=param.values[f'units_{i}'], activation=param.values['activation']))

# Output layer
nn_op.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn_op.summary()

In [ ]:
# Compile the model
nn_op.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
history = nn_op.fit(X_train_scaled, y_train, epochs=100, verbose=1, batch_size=X_train_scaled.shape[1])

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_op.evaluate(X_test_scaled,y_test,verbose=1, batch_size=1)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export our model to HDF5 file
nn_op.save('AlphabetSoupCharity_Optimization.h5')